In [1]:
import pandas as pd

In [2]:
df_games = pd.read_csv("steam_games_clean.csv")

In [3]:
df_games.head()

,genres,title,price,id,developer,release_year
0,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,4.99,761140.0,Kotoshiro,2018
1,"['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,0.00,643980.0,Secret Level SRL,2018
2,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",Real Pool 3D - Poolians,0.00,670290.0,Poolians.com,2017
3,"['Action', 'Adventure', 'Casual']",弹炸人2222,0.99,767400.0,彼岸领域,2017
4,"['Action', 'Adventure', 'Simulation']",Battle Royale Trainer,3.99,772540.0,Trickjump Games Ltd,2018


In [5]:
#Ordeno las columnas
ordered_columns = ["title","id","developer","release_year","genres","price"]
df_games = df_games[ordered_columns]

In [ ]:
#Hago el encoding de las columnas de género para hacer el sistema.\n",
import ast
df_games["genres"] = df_games["genres"].apply(lambda x: ast.literal_eval(x))
genres = pd.DataFrame(df_games["genres"].tolist())\n
genres_obj = genres.stack()
genres_df = pd.get_dummies(genres_obj)
genres_df = genres_df.groupby(level = [0], axis = 0).sum()

In [7]:
genres_df.sum().sort_values(ascending=False)

Indie                        15123
Action                       10697
Casual                        8061
Adventure                     7809
Strategy                      6701
Simulation                    6511
RPG                           5121
Free to Play                  1881
Early Access                  1383
Sports                        1206
Racing                        1049
Massively Multiplayer         1000
Design &amp; Illustration      444
Utilities                      315
Web Publishing                 257
Animation &amp; Modeling       163
Education                      118
Software Training              102
Video Production               101
Audio Production                84
Photo Editing                   77
Accounting                       7
dtype: int64

In [8]:
genres_df = genres_df[["Indie","Action","Casual","Adventure","Strategy","Simulation","RPG","Free to Play","Sports","Racing"]]

In [9]:
df_games = pd.concat([df_games, genres_df], axis = 1)
df_games = df_games.drop(columns="genres")

In [10]:
df_games.columns

Index(['title', 'id', 'developer', 'release_year', 'price', 'Indie', 'Action',
       'Casual', 'Adventure', 'Strategy', 'Simulation', 'RPG', 'Free to Play',
       'Sports', 'Racing'],
      dtype='object')

In [11]:
#Selecciono las primeras 5000 filas para que mi pc de 8gb de ram pueda procesarlo.

df_games = df_games.head(5000)

In [13]:
from sklearn.metrics.pairwise import cosine_similarity

def calcular_recomendacion(df_games, title, top_n=5):

    features = df_games[['Indie', 'Action', 'Casual', 'Adventure', 'Strategy', 'Simulation', 'RPG', 'Free to Play', 'Sports', 'Racing']]

    similarity_matrix = cosine_similarity(features)

    indice_juego = df_games[df_games['title'] == title].index[0]

    similar_juegos = list(enumerate(similarity_matrix[indice_juego]))

    similar_juegos = sorted(similar_juegos, key=lambda x: x[1], reverse=True)

    top_n_similar_juegos = similar_juegos[1:top_n + 1]

    top_n_indices = [x[0] for x in top_n_similar_juegos]

    juegos_recomendados = df_games.iloc[top_n_indices]['title'].tolist()

    return juegos_recomendados

In [15]:
juegos = df_games["title"]

recomendaciones_dict = {}

for title in juegos:
    recomendaciones = calcular_recomendacion(df_games, title, top_n=5)
    recomendaciones_dict[title] = recomendaciones

recomendaciones_df = pd.DataFrame(recomendaciones_dict.items(), columns=["Juego", "Juegos Recomendados"])

In [17]:
recomendaciones_df.to_csv("sistema_recomendacion.csv",index=False)

In [18]:
recomendaciones_df

,Game,Recommended Games
0,Lost Summoner Kitty,"[Pixel Puzzles 2: Anime, World of Cinema - Dir..."
1,Ironbound,"[Shadow Hunter, Immortal Empire, Tactical Craf..."
2,Real Pool 3D - Poolians,"[SNOW - All Access Basic Pass, SNOW - All Acce..."
3,弹炸人2222,"[Atomic Adam: Episode 1, Biozone, Luxor: 5th P..."
4,Battle Royale Trainer,"[The Tomorrow War, Beyond Space Remastered Edi..."
...,...,...
4995,Disgaea PC / 魔界戦記ディスガイア PC,"[Rise of the Argonauts, Drakensang, Guild Wars..."
4996,24 Hours 'til Rescue: Cheat Pack!,"[Fearless Fantasy, Luna: Shattered Hearts: Epi..."
4997,Disgaea PC / 魔界戦記ディスガイア PC - Digital Art Book ...,"[Rise of the Argonauts, Drakensang, Guild Wars..."
4998,軒轅劍外傳穹之扉音樂精選集(Sound Collection of Xuan-Yuan Sw...,"[Rise of the Argonauts, Drakensang, Guild Wars..."
